# ASMSA: Tune AAE model hyperparameters

**Previous step**
- [prepare.ipynb](prepare.ipynb): Download and sanity check input files

**Next steps**
- [train.ipynb](train.ipynb): Use results of previous tuning in more thorough training
- [md.ipynb](md.ipynb): Use a trained model in MD simulation with Gromacs

## Notebook setup

In [ ]:
threads = 2
import os
os.environ['OMP_NUM_THREADS']=str(threads)
import tensorflow as tf

# PyTorch favours OMP_NUM_THREADS in environment
import torch

# Tensorflow needs explicit cofig calls
tf.config.threading.set_inter_op_parallelism_threads(threads)
tf.config.threading.set_intra_op_parallelism_threads(threads)

In [ ]:
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import mdtraj as md
import numpy as np
import urllib.request
from tensorflow import keras
import keras_tuner
import asmsa
from datetime import datetime
import tensorflow as tf

## Input files

All input files are prepared (up- or downloaded) in [prepare.ipynb](prepare.ipynb). 

This is for demonstration purpose, in real use the inputs should be placed here, and _conf, traj, topol, index_ variables set to their filenames names.

In [ ]:
# Define input files

# input conformation
#conf = "alaninedipeptide_H.pdb"
conf = "trpcage_correct.pdb"

# input trajectory
# atom numbering must be consistent with {conf}

#traj = "alaninedipeptide_reduced.xtc"
traj = "trpcage_red.xtc"

# input topology
# expected to be produced with 
#    gmx pdb2gmx -f {conf} -p {topol} -n {index} -o {gro}

# Gromacs changes atom numbering, the index file must be generated and used as well
# gro file is used to generate inverse indexing for plumed.dat

#topol = "topol.top"
topol = "topol_correct.top"
index = 'index_correct.ndx'
gro = 'trpcage_correct.gro'

## Load dataset

In [ ]:
# load train dataset
X_train = tf.data.Dataset.load('datasets/intcoords/train')
X_train_np = np.stack(list(X_train))
X_train_np = X_train_np[:int(0.5*len(X_train_np))]

# load validation dataset
X_validate = tf.data.Dataset.load('datasets/intcoords/validate')
X_validate_np = np.stack(list(X_validate))

X_train_np.shape, X_validate_np.shape

## Hyperparameter definition
Specify hyperparameter ranges

In [ ]:
medium_hp = {
    'activation' : ['relu','gelu','selu'],
    'ae_neuron_number_seed' : [32,96,128],
    'disc_neuron_number_seed' : [32,96],
    'ae_number_of_layers' : [2,5],
    'disc_number_of_layers' : [2,5],
    'batch_size' : [64,128,256,512],
    'optimizer' : ['Adam'],
    'learning_rate' : 0.0002,
    'ae_loss_fn' : ['MeanSquaredError'],
    'disc_loss_fn' : ['BinaryCrossentropy']
}

tiny_hp = {
    'activation' : ['relu'],
    'ae_neuron_number_seed' : [32,96],
    'disc_neuron_number_seed' : [32,96],
    'ae_number_of_layers' : [2,2],
    'disc_number_of_layers' : [3,3],
    'batch_size' : [64,128,256],
    'optimizer' : ['Adam'],
    'learning_rate' : 0.0002,
    'ae_loss_fn' : ['MeanSquaredError'],
    'disc_loss_fn' : ['BinaryCrossentropy']
}

## Sequential hyperparameter tuning

This is robust, it does not require Kubernetes environment for additional job submission. On the other hand, it is slow accordingly.

**Skip to the next section if you run the notebook in our recommended setup in Kubernets.**

In [ ]:
# Just testing numbers of epochs and hyperparameter setting trials
# Don't expect anything meaningful
trials=4
epochs=100

# Set RESULTS_DIR env variable for results of tuning
os.environ['RESULTS_DIR'] = datetime.today().strftime("%m%d%Y-%H%M%S")
tuner = keras_tuner.RandomSearch(
    max_trials=trials,
    hypermodel=
        asmsa.AAEHyperModel(
            (X_validate_np.shape[1],),
            hp=medium_hp,
            prior=tfp.distributions.Normal(loc=1, scale=1)),
    objective=keras_tuner.Objective("score", direction="min"),
    directory="./results",
    project_name="Random",
    overwrite=True
)

In [ ]:
tuner.search(train=X_train_np,validation=X_validate_np,epochs=epochs,verbose=2)

In [ ]:
from asmsa.tuning_analyzer import TuningAnalyzer

# Create analyzer object that analyses results of tuning
# By default it is the latest tuning, but can by choosen with tuning flag,
#  e.g TuningAnalyzer(tuning='analysis/05092023-135249')
analyzer = TuningAnalyzer()

In [ ]:
# Get sorted hyperparameters by score, by default 10 best HP, for different number:
#  analyzer.get_best_hp(num_trials=3)
analyzer.get_best_hp()

In [ ]:
# Matplotlib visualization - not recommended way, does not look that good and does not scale 
#  that well but at least the colors are consistent accross measures. After more work could look better
# - By default visualizing best 10 trials
# - Can specify only one specific trial... analyzer.visualize_tuning(trial='15d9fa928a7517004bcb28771bb6e5f17ad66dd7013c6aa1572a84773d91393c')
# - Can specify number of best trials to be visualized... analyzer.visualize_tuning(num_trials=3)
analyzer.visualize_tuning()

In [ ]:
# Recommended option via Tensorboard. This function populates TB event
#  which can be viewed in native way via Tensorboard. By default chooses
#  latest tuning and populates into its directory _TB, e.g: analysis/05092023-135249/_TB
# - Can override directory where to populate... analyzer.populate_TB(out_dir='MyTBeventDir')
# - Can choose only specific trials via list... analyzer.populate_TB(trials=['15d9fa928a7517004bcb28771bb6e5f17ad66dd7013c6aa1572a84773d91393c']),
# - Can select how many best trials to be visualized... analyzer.populate_TB(num_trials=3)
analyzer.populate_TB(num_trials=3)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir analysis

## Parallel hyperparameter tuning

In [ ]:
# Finally, this is the real stuff
# medium settings known to be working for trpcage

epochs=15
trials=3
hp=medium_hp

# testing only
#epochs=8
#trials=6
#hp=tiny_hp

In [ ]:
# number of parallel workers, each runs a single trial at time
# balance between resource availability and size of the problem
# currently each slave runs on 4 cores and 4 GB RAM (hardcoded in src/asmsa/tunewrapper.py)

slaves=3

In [ ]:
# XXX: Kubernetes magic: find out names of container image and volume
# check the result, it can go wrong

with open('IMAGE') as img:
    image=img.read().rstrip()

import re
mnt=os.popen('mount | grep /home/jovyan').read()
pvcid=re.search('pvc-[0-9a-z-]+',mnt).group(0)
pvc=os.popen(f'kubectl get pvc | grep {pvcid} | cut -f1 -d" "').read().rstrip()

print(f"""\
image: {image}
volume: {pvc}
""")

In [ ]:
# Python wrapper around scripts that prepare and execute parellel Keras Tuner in Kubernetes
from asmsa.tunewrapper import TuneWrapper

wrapper = TuneWrapper(ds=X_validate_np,hp=hp,output=datetime.today().strftime("%m%d%Y-%H%M%S"),epochs=epochs,trials=trials,pdb=conf,top=topol,xtc=traj,ndx=index, pvc=pvc)

In [ ]:
# Necessary but destructive cleanup before hyperparameter tuning

# DON'T RUN THIS CELL BLINDLY
# it kills any running processes including the workers, and it purges previous results

!kubectl delete job/tuner
!kill $(ps ax | grep tuning.py | awk '{print $1}')
!rm -rf results

In [ ]:
# start the master (chief) of tuners in background
# the computation takes rather long, this is a more robust approach then keeping it in the notebook

wrapper.master_start()

In [ ]:
# therefore one should check the status ocassionally; it should show a tuning.py process running
print(wrapper.master_status())

In [ ]:
# spawn the requested number of workers as separate Kubernetes job with several pods 
# they receive work from 

wrapper.workers_start(num=slaves)

In [ ]:
# This status should show {slaves} number of pods, all of them start in Pending state, and follow through ContainerCreating 
# to Running, and Completed finally

# This takes time, minutes to hours depending on size of the model, number of trials, and number of slaves
# Run this cell repeatedly, waiting until all the pods are completed

wrapper.workers_status()

In [ ]:
# Same steps for analysis as with serial tuning
analyzer = TuningAnalyzer()
analyzer.get_best_hp()

In [ ]:
# We can choose output dir for TB event this time
out = 'dist_tuning'

analyzer.populate_TB(out_dir=out)

In [ ]:
# Might need to kill previous tensorboard instance to change logdir
!pkill -f 'tensorboard'

%load_ext tensorboard
%tensorboard --logdir $out